<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 4.2: FIRRTL AST Traversal

**Prev: [Introduction to FIRRTL](4.1_firrtl_ast.ipynb)**<br>
**Next: [Common Pass Idioms](4.3_firrtl_common_idioms.ipynb)**

### Understanding IR node children

编写一个 Firrtl pass 通常需要编写函数来遍历 Firrtl 数据结构，以收集信息或用新 IR 节点替换旧 IR 节点。

IR 数据结构是一棵树，其中每个 IR 节点可以有一些子节点（这些子节点又可以有更多子节点，等等）。没有子节点的 IR 节点称为叶节点。

不同的 IR 节点可以具有不同的子节点类型。下表显示了每个 IR 节点类型的可能子节点类型：

```
+------------+-----------------------------+
|    Node    |          Children           |
+------------+-----------------------------+
| Circuit    | DefModule                   |
| DefModule  | Port, Statement             |
| Port       | Type, Direction             |
| Statement  | Statement, Expression, Type |
| Expression | Expression, Type            |
| Type       | Type, Width                 |
| Width      |                             |
| Direction  |                             |
+------------+-----------------------------+
```

### The map function

要编写一个遍历 `Circuit` 的函数，我们首先需要了解函数式编程概念 `map`。

#### Understanding Seq.map
一个字符串序列，可以表示为一个根节点为 `Seq`，子节点为 `"a"`、`"b"` 和 `"c"` 的树：
```scala
val s = Seq("a", "b", "c")
```
```
    Seq
 /   |   \
"a" "b" "c"
```

假设我们定义一个函数 `f`，给定一个字符串参数 `x`，将 `x` 与自身连接起来：
```scala
def f(x: String): String = x + x
```

我们可以调用 `s.map` 来返回一个新的 `Seq[String]`，其子节点是将 `f` 应用于 s 的每个子节点的结果：
```scala
val s = Seq("a", "b", "c")
def f(x: String): String = x + x  // repeated declaration for clarity
val t = s.map(f)
println(t) // Seq("aa", "bb", "cc")
```
```
     Seq
 /    |    \
"aa" "bb" "cc"
```

#### Understanding Firrtl's map

我们使用这种“映射”思想在 IR 节点上创建我们自己的自定义 `map` 方法。假设我们有一个表示 1 + 1 的 `DoPrim` 表达式；这可以描绘为一个根节点为 `DoPrim` 的表达式树：
```
        DoPrim
     /          \
UIntValue    UIntValue
```

如果我们有一个函数 `f`，它接受一个 `Expression` 参数并返回一个新的 `Expression`，我们可以将其“映射”到给定 IR 节点（如我们的 `DoPrim`）的所有子 `Expression` 上。这将返回以下新的 `DoPrim`，其子节点是将 `f` 应用于 `DoPrim` 的每个 `Expression` 子节点的结果：
```
        DoPrim
     /          \
f(UIntValue)    f(UIntValue)
```

有时 IR 节点有多种类型的子节点。例如，`Conditionally` 既有 `Expression` 子节点也有 `Statement` 子节点。在这种情况下，map 将只将其函数应用于类型匹配函数参数类型（和返回值类型）的子节点：
```scala
val c = Conditionally(info, e, s1, s2) // e: Expression, s1, s2: Statement, info: FileInfo
def fExp(e: Expression): Expression = ...
def fStmt(s: Statement): Statement = ...
c.map(fExp)  // Conditionally(fExp(e), s1, s2)
c.map(fStmt) // Conditionally(e, fStmt(s1), fStmt(s2))
```

Scala 有“中缀表示法”，当调用只有一个参数的函数时，可以省略 `.` 和括号。通常，我们使用中缀表示法编写这些 map 函数：
```scala
c map fExp  // equivalent to c.map(fExp)
c map fStmt // equivalent to c.map(fStmt)
```

### Pre-order traversal

要遍历 Firrtl 树，我们使用 `map` 编写递归函数，以访问我们关心的每个节点的每个子节点。

假设我们想收集设计中声明的每个寄存器的名称；我们知道这需要访问每个 `Statement`。然而，一些 `Statement` 节点可以有子 `Statement`。因此，我们需要编写一个函数，它既检查其输入参数是否为 `DefRegister`，如果不是，则递归应用 `f` 于其输入参数的所有 `Statement` 子节点：

以下函数 `f` 与我们描述的函数类似，但它有两个参数：一个可变的寄存器名称哈希集和一个 `Statement`。使用函数柯里化，我们可以只传递第一个参数来返回一个具有所需类型签名的新函数（`Statement=>Statement`）：

```scala
def f(regNames: mutable.HashSet[String]())(s: Statement): Statement = s match {
  // If register, add name to regNames
  case r: DefRegister =>
    regNames += r.name
    r // Return argument unchanged (ok because DefRegister has no Statement children)
  // If not, apply f(regNames) to all children Statement
  case _ => s map f(regNames) // Note that f(regNames) is of type Statement=>Statement
}
```

这种模式在 Firrtl 中非常常见，被称为“前序遍历”，因为递归函数在递归应用于子节点之前匹配原始 IR 节点。

### Post-order traversal

我们可以如下编写先前示例中的“后序遍历”：

```scala
def f(regNames: mutable.HashSet[String]())(s: Statement): Statement = 
  // Not we immediately recurse to the children nodes, then match
  s map f(regNames) match {
    // If register, add name to regNames
    case r: DefRegister =>
      regNames += r.name
      r // Return argument unchanged (ok because DefRegister has no Statement children)
    // If not, return s
    case _ => s // Note that all Statement children of s have had f(regNames) already applied
  }
```

虽然这两个示例之间的遍历顺序不同，但对于这个用例（以及许多其他用例）没有区别。然而，当遍历顺序很重要时，这是一个重要的工具。